In [9]:
import requests
import geopandas as gpd
import osmnx as ox
import matplotlib.pyplot as plt
from shapely.geometry import Point

# Get data on Japanese restaurants
japanese_restaurants = requests.get(
    "https://data.cityofnewyork.us/resource/43nn-pn8j.json?cuisine_description=Japanese&$limit=2000"
).json()

# Convert to GeoDataFrame
japanese_restaurants_gdf = gpd.GeoDataFrame(
    japanese_restaurants,
    geometry=gpd.points_from_xy(
        [float(x["Longitude"]) for x in japanese_restaurants],
        [float(x["Latitude"]) for x in japanese_restaurants],
    ),
)

japanese_restaurants_gdf.fillna("na", inplace=True)
japanese_restaurants_gdf.drop_duplicates(subset="camis", inplace=True)


KeyError: 'Longitude'

In [10]:
# Get the bounding box
bbox = japanese_restaurants_gdf.total_bounds

# Create a network of cities
city_network = ox.graph_from_bbox(
    bbox[3], bbox[1], bbox[2], bbox[0], network_type="walk"
)

# Convert network to GeoDataFrame
network_nodes, network_edges = ox.graph_to_gdfs(city_network)


NameError: name 'japanese_restaurants_gdf' is not defined

In [11]:
# Define the starting point
start_point = Point(-73.9614755, 40.8081037)  

# Get the network node of the starting point
start_node = ox.distance.nearest_nodes(city_network, start_point.x, start_point.y)

# Choose a Japanese restaurant at random as a destination
destination = japanese_restaurants_gdf.sample(1)
destination_node = ox.distance.nearest_nodes(city_network, destination.geometry.x.iloc[0], destination.geometry.y.iloc[0])

# Calculate the shortest path
route = ox.shortest_path(city_network, start_node, destination_node, weight="length")

# Mapping the shortest path
fig, ax = ox.plot_graph_route(city_network, route, node_size=0)


NameError: name 'city_network' is not defined

In [ ]:
def distance_from_point(row, origin):
    dest = ox.distance.nearest_nodes(city_network, row.geometry.x, row.geometry.y)
    route = ox.shortest_path(city_network, origin, dest, weight="length")
    edge_lengths = ox.routing.route_to_gdf(city_network, route)["length"]
    route_geom = ox.routing.route_to_gdf(city_network, route)
    route_geom = route_geom.geometry.unary_union
    return {"distance": sum(edge_lengths) / 1609.34, "route_geom": route_geom}

# Calculate the distance
japanese_restaurants_gdf["distance_from_start"] = japanese_restaurants_gdf.apply(
    distance_from_point, origin=start_node, axis=1
)

# Convert results to GeoDataFrame
japanese_restaurants_routes = gpd.GeoDataFrame(japanese_restaurants_gdf, geometry="route_geom", crs="EPSG:4326")

# Mapping
ax = gpd.GeoSeries(start_point).plot(color="black")
network_edges.plot(ax=ax, color="black", linewidth=0.1)
japanese_restaurants_routes.plot(column="distance", cmap="magma", ax=ax, linewidth=3)
japanese_restaurants_gdf.plot(column="distance", cmap="magma", legend=True, ax=ax).set_axis_off()
